In [2]:
from pathlib import Path

In [3]:
import sys
sys.path.insert(0,str(Path('mitchu').absolute()))
from mitchu.lib import *
from mitchu.metrics import *

In [ ]:
class sexual_act(Action):

    def __init__(self, minT=0.5, maxT=10, alpha=0.1): # *args,**kwargs
        if minT >= maxT:
            raise Exception("minT >= maxT")
        
        self.minT = minT
        self.maxT = maxT
        self.alpha = alpha
        
        super().__init__()

    
    def act_time(self):
        if self.person.hours_per_week() == 0: return self.minT
        if self.person.hours_per_week() >= self.person.args['extraversion']: return self.maxT
        
        step1 = mu( 
            x=self.person.args['extraversion'] / self.person.hours_per_week() - 1,
            alpha=self.alpha, # alpha
            minT=self.minT,
            maxT=self.maxT
        )
        return step1
    
    def act(self, context):
        self.log("ACT_BEGIN!", context)
        from random import random, choice
        # go out to make a friend.
        # I decide to go do it...
        self.log("trying to make a friend", context)
        
        iwant = self.person.args['extraversion'] - self.person.hours_per_week()
        if iwant < 0:
            # this shouldn't be possible
            return
        
        if random() > 0.8:
            self.log("randomly failed at making a friend", context)
            return
            
        potential_friends = {
            p for p in context.ppl
            if p != self.person and \
                p not in self.person.friends
            and p.hours_per_week() < p.args['extraversion']
        }
        
        #print([
        #    (p, self.person.friends) for p in context.ppl
        #    if p != self.person and \
        #        p not in self.person.friends
        #])
        potential_friends = list(potential_friends)
        
        if not len(potential_friends):
            self.log("No friends available... sad", context)
            return
            
        my_new_friend = choice(potential_friends)
        
        theywant = my_new_friend.args['extraversion'] - my_new_friend.hours_per_week()
        
        time_agree = np.random.uniform(iwant, theywant)
        
        my_new_friend.friends[self.person] = time_agree
        self.person.friends[my_new_friend] = time_agree
        
        self.log("Made friend w/ %s" % my_new_friend.name, context)
        
class Person(Person):
    
    def hours_per_week(self):
        return sum(list(self.friends.values()))
        
    def __init__(self, *args, **kwargs):
        self.friends = {}
        
        super().__init__(*args,**kwargs)

In [ ]:
models = []

for i in range(10):
    print("%s/%s" % (i,10))
    people = [ Person(extraversion=float(i)/150+0.5) for i in range(300) ]

    [p.addact([
        make_friend(minT=2,maxT=100),
        end_friendship(maxT=100,minT=2)
    ]) for p in people];

    m = sim(people, debug=False)
    m.addmet( 
        degree_distribution,
        unhappiness_max,
        unhappiness_avg,
        network,
        percent_oversocialized,
        percent_lonely
    )
    m.run(100, 0.1)
    models.append(m)
